# End to end First Machine Learning Project

## Task 1: Install the Azure ML SDK for Python
The Azure ML SDK for Python provides classes you can use to work with Azure ML in your Azure subscription. Run the cell below to install the azureml-sdk Python package, including the optional notebooks component; which provides some functionality specific to the Jupyter Notebooks environment.

More Information: For more details about installing the Azure ML SDK and its optional components, see the Azure ML SDK Documentation

In [6]:
#!pip install azureml.core

import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.19.0


In [7]:
from azureml.core import authentication

auth = authentication.InteractiveLoginAuthentication()
print('Signed in')

Signed in


## Read in config file
This saved configuration file enables you to easily obtain a reference to the workspace by simply loading it, as demonstrated in the following cell. Note that this method will prompt you to reauthenticate against your Azure subscription if your session has expired.

In [8]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with amlworkspacesahiep2141


# Run your first Experiment

In [9]:
from azureml.core import Experiment, Run
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-experiment")
print("Starting experiment:", experiment.name)

# Start logging data from the experiment
run = experiment.start_logging()

# load the diabetes dataset
data = pd.read_csv('data/diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log("observations", row_count)

# Create box plots for each feature variable by the "diabetic" label and log them
num_cols = data.columns[:-1]
for col in num_cols:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    data.boxplot(column = col, by = "Diabetic", ax = ax)
    ax.set_title(col + ' by Diabetic')
    ax.set_ylabel(col)
    run.log_image(name = col, plot = fig)
plt.show()

# Create a list of mean diabetes pedigree per age and log it
mean_by_age = data[["Age", "DiabetesPedigree"]].groupby(["Age"]).mean().reset_index()
ages = mean_by_age["Age"].tolist()
pedigrees = mean_by_age["DiabetesPedigree"].tolist()
for index in range(len(ages)):
       run.log_row("Mean Diabetes Pedigree by Age", Age = ages[index],Diabetes_Pedigree = pedigrees[index])

# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv("sample.csv", index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = './sample.csv')

# Complete tracking and get link to details
run.complete()

Starting experiment: diabetes-experiment


FileNotFoundError: [Errno 2] File b'data/diabetes.csv' does not exist: b'data/diabetes.csv'

In [10]:
# View Experiment Result
run

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes-experiment,2aa90723-b2da-4233-b114-c5de57c68705,,Running,Link to Azure Machine Learning studio,Link to Documentation
